In [1]:
import random

In [2]:
rows = '1234'
cols = '1234'
boxes = [r+c for r in rows for c in cols]
baseline_box_to_number = {k: v for k, v in zip(boxes, list(range(1, 16)) + [0])}
baseline_number_to_box = {v: k for k, v in baseline_box_to_number.items()}

In [3]:
def blank_box(values):
    for box, number in values.items():
        if number == 0:
            return box
    raise Exception('Invalid state, blank box not found!')

def manhattan_distance(box, number):
    expected_box =  baseline_number_to_box[number]
    return abs(int(box[0]) - int(expected_box[0])) + \
           abs(int(box[1]) - int(expected_box[1]))

def hcost(values):
    wrong_boxes = [b for b in boxes if values[b] != baseline_box_to_number[b]]
    # Heuristic: Wrong Position
    cost1 = len(wrong_boxes)
    # Heuristic 2: Total distance
    cost2 = sum(manhattan_distance(b, values[b]) for b in wrong_boxes)
    return max(cost1, cost2)

def display(values):
    print('     1  2  3  4')
    print('---------------')
    lines = [' '.join([v if len(v)==2 else ' ' + ('*' if v == '0' else v) for v in [str(values[r+c]) for c in cols]]) for r in rows]
    print('1 | ' + lines[0])
    print('2 | ' + lines[1])
    print('3 | ' + lines[2])
    print('4 | ' + lines[3])

def available_moves(values):
    bb = blank_box(values)
    x, y = int(bb[0]), int(bb[1])
    moves = []
    b = str(x - 1) + str(y)
    if b in boxes: moves.append('Move from top (↓)')
    b = str(x + 1) + str(y)
    if b in boxes: moves.append('Move from bottom (↑)')
    b = str(x) + str(y - 1)
    if b in boxes: moves.append('Move from left (→)')
    b = str(x) + str(y + 1)
    if b in boxes: moves.append('Move from right (←)')
    return moves

def move_slide(values, move_type):
    new_values = values.copy()
    bb = blank_box(values)
    x, y = int(bb[0]), int(bb[1])
    if move_type == 'Move from top (↓)':
        tb = str(x - 1) + str(y)
    elif move_type == 'Move from bottom (↑)':
        tb = str(x + 1) + str(y)
    elif move_type == 'Move from left (→)':
        tb = str(x) + str(y - 1)
    elif move_type == 'Move from right (←)':
        tb = str(x) + str(y + 1)
    else:
        raise Exception('Invalid move type - "{}"'.format(move_type))
        
    if tb not in boxes:
        raise Exception('Unable to proceed with move type - "{}"'.format(move_type))
        
    new_values[bb] = new_values[tb]
    new_values[tb] = 0
    return new_values

def random_puzzle(random_moves=10):
    puzzle = baseline_box_to_number.copy()
    for _ in range(random_moves):
        next_move = random.choices(available_moves(puzzle))[0]
        puzzle = move_slide(puzzle, next_move)
    return puzzle

def hash_puzzle(values):
    return '-'.join([str(values[b]) for b in boxes])

def solve_puzzle(puzzle, print_result=True):
    if hcost(puzzle)==0:
        return 0
    
    #################################
    # Prepare root node to tranverse
    ################################
    root_id = hash_puzzle(puzzle) 
    pending_nodes = {
        root_id: {
            'id': root_id,
            'state': puzzle,
            'last_move': None,
            'parent': None,
            'cost': hcost(puzzle)
        }
    }
    best_node = pending_nodes[root_id]
    visited_ids = set([root_id])
    
    ###############################
    # A* Search, choose min cost
    ###############################
    while pending_nodes:
        ########################
        # Termination Condition
        ########################
        if best_node['cost'] == 0:
            if print_result:
                print('------- Start -------')
                print()

            solution_path = [best_node]

            while best_node['parent']:
                best_node = best_node['parent']
                solution_path.append(best_node)

            solution_path = solution_path[::-1]
            for index, node in enumerate(solution_path):
                if print_result:
                    if node['last_move']:
                        print()
                        print('Step {}: {}, Cost = {}'.format(index, node['last_move'], node['cost']))
                        print()

                    display(node['state'])
            
            if print_result:
                print()
                print('------- Done -------')
            
            return index
        
        ############################
        # Get possible next puzzles
        ############################
        current_puzzle = best_node['state']
        current_id = best_node['id']
        for move_type in available_moves(current_puzzle):
            next_puzzle = move_slide(current_puzzle, move_type)
            next_id = hash_puzzle(next_puzzle)
            if next_id not in visited_ids:
                pending_nodes[next_id] = {
                    'id': next_id,
                    'state': next_puzzle,
                    'last_move': move_type,
                    'parent': best_node,
                    'cost': hcost(next_puzzle)
                }
                visited_ids.add(next_id)
        
        ######################
        # Delete best node
        ######################
        del pending_nodes[current_id]
        
        ######################
        # Find next best node
        ######################
        _, best_node = min([(node['cost'], node) for _id, node in pending_nodes.items()], 
                             key=lambda x: x[0])
        
def sliding_puzzle_simulation(random_moves):
    required_moves = solve_puzzle(random_puzzle(random_moves))
    if required_moves == 0:
        return
    
    print()
    print('Number of moves to generate puzzle: {}'.format(random_moves))
    print('Number of moves to solve puzzle: {}'.format(required_moves))
    print('Efficiency of algorithm = {:.2f}'.format(random_moves / required_moves))

In [4]:
sliding_puzzle_simulation(30)

------- Start -------

     1  2  3  4
---------------
1 |  1  2  3  4
2 |  9  5  6  7
3 | 13 10 11  8
4 | 14 15 12  *

Step 1: Move from left (→), Cost = 10

     1  2  3  4
---------------
1 |  1  2  3  4
2 |  9  5  6  7
3 | 13 10 11  8
4 | 14 15  * 12

Step 2: Move from left (→), Cost = 10

     1  2  3  4
---------------
1 |  1  2  3  4
2 |  9  5  6  7
3 | 13 10 11  8
4 | 14  * 15 12

Step 3: Move from left (→), Cost = 10

     1  2  3  4
---------------
1 |  1  2  3  4
2 |  9  5  6  7
3 | 13 10 11  8
4 |  * 14 15 12

Step 4: Move from top (↓), Cost = 10

     1  2  3  4
---------------
1 |  1  2  3  4
2 |  9  5  6  7
3 |  * 10 11  8
4 | 13 14 15 12

Step 5: Move from top (↓), Cost = 10

     1  2  3  4
---------------
1 |  1  2  3  4
2 |  *  5  6  7
3 |  9 10 11  8
4 | 13 14 15 12

Step 6: Move from right (←), Cost = 8

     1  2  3  4
---------------
1 |  1  2  3  4
2 |  5  *  6  7
3 |  9 10 11  8
4 | 13 14 15 12

Step 7: Move from right (←), Cost = 6

     1  2  3  4
-----------

In [5]:
sliding_puzzle_simulation(20)

------- Start -------

     1  2  3  4
---------------
1 |  2  3  *  4
2 |  1  6  7  8
3 |  5  9 11 12
4 | 13 10 14 15

Step 1: Move from left (→), Cost = 12

     1  2  3  4
---------------
1 |  2  *  3  4
2 |  1  6  7  8
3 |  5  9 11 12
4 | 13 10 14 15

Step 2: Move from left (→), Cost = 12

     1  2  3  4
---------------
1 |  *  2  3  4
2 |  1  6  7  8
3 |  5  9 11 12
4 | 13 10 14 15

Step 3: Move from bottom (↑), Cost = 10

     1  2  3  4
---------------
1 |  1  2  3  4
2 |  *  6  7  8
3 |  5  9 11 12
4 | 13 10 14 15

Step 4: Move from bottom (↑), Cost = 8

     1  2  3  4
---------------
1 |  1  2  3  4
2 |  5  6  7  8
3 |  *  9 11 12
4 | 13 10 14 15

Step 5: Move from right (←), Cost = 6

     1  2  3  4
---------------
1 |  1  2  3  4
2 |  5  6  7  8
3 |  9  * 11 12
4 | 13 10 14 15

Step 6: Move from bottom (↑), Cost = 4

     1  2  3  4
---------------
1 |  1  2  3  4
2 |  5  6  7  8
3 |  9 10 11 12
4 | 13  * 14 15

Step 7: Move from right (←), Cost = 2

     1  2  3  4
-----

In [6]:
sliding_puzzle_simulation(10)

------- Start -------

     1  2  3  4
---------------
1 |  1  2  3  4
2 |  5  6  8 11
3 |  9 10  7 12
4 | 13  * 14 15

Step 1: Move from right (←), Cost = 6

     1  2  3  4
---------------
1 |  1  2  3  4
2 |  5  6  8 11
3 |  9 10  7 12
4 | 13 14  * 15

Step 2: Move from right (←), Cost = 4

     1  2  3  4
---------------
1 |  1  2  3  4
2 |  5  6  8 11
3 |  9 10  7 12
4 | 13 14 15  *

Step 3: Move from top (↓), Cost = 6

     1  2  3  4
---------------
1 |  1  2  3  4
2 |  5  6  8 11
3 |  9 10  7  *
4 | 13 14 15 12

Step 4: Move from top (↓), Cost = 6

     1  2  3  4
---------------
1 |  1  2  3  4
2 |  5  6  8  *
3 |  9 10  7 11
4 | 13 14 15 12

Step 5: Move from left (→), Cost = 6

     1  2  3  4
---------------
1 |  1  2  3  4
2 |  5  6  *  8
3 |  9 10  7 11
4 | 13 14 15 12

Step 6: Move from bottom (↑), Cost = 4

     1  2  3  4
---------------
1 |  1  2  3  4
2 |  5  6  7  8
3 |  9 10  * 11
4 | 13 14 15 12

Step 7: Move from right (←), Cost = 2

     1  2  3  4
-------------